Importing Important Files

In [17]:
import qiskit as qt
import numpy as np
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit import Aer
from qiskit.opflow import PauliExpectation, CircuitSampler, StateFn, AerPauliExpectation, CircuitStateFn, CircuitOp, MatrixExpectation
from qiskit_nature.circuit.library import HartreeFock
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.converters.second_quantization import QubitConverter
import matplotlib.pyplot as plt

Creating a function that would measure the expectation $<\Psi|\hat{O}|\Psi>$

In [18]:

def expectation(operator, state):
    backend_i = Aer.get_backend('statevector_simulator')
    psi = CircuitStateFn(state)
    measurable_expression = StateFn(operator, is_measurement=True).compose(psi)
    expectation_i = PauliExpectation().convert(measurable_expression)
    sampler = CircuitSampler(backend_i).convert(expectation_i)
    electronic_energy = np.real(sampler.eval())
    return electronic_energy

Obtaining the molecular Hamiltonian in second quantised notation using PySCF Driver

In [19]:
coord = 'H 0.0 0.0 0.0; H 0.0 0.0 0.735'
driver = PySCFDriver(atom=coord, charge=0, spin=0, basis='sto3g')
es_problem = ElectronicStructureProblem(driver)
second_q_op = es_problem.second_q_ops()

Converting the Hamiltonian in terms of Pauli Gates using Jordan-Wigner Mapping

In [20]:
mapper = JordanWignerMapper()
converter = QubitConverter(mapper=mapper, two_qubit_reduction=False)
qubit_op = converter.convert(second_q_op['ElectronicEnergy'])

Obtaining some properties of the molecule such as number of electrons, number of spin-orbitals in a given basis

In [21]:
es_particle_number = es_problem.grouped_property_transformed.get_property('ParticleNumber')
num_particles = (es_particle_number.num_alpha, es_particle_number.num_beta)
num_spin_orbitals = es_particle_number.num_spin_orbitals
es_energy = es_problem.grouped_property_transformed.get_property('ElectronicEnergy')
nuclear_repulsion_energy = es_energy.nuclear_repulsion_energy
print('Number Of Particles: ', num_particles)
print('Number of Spin Orbitals: ', num_spin_orbitals)
print('Nuclear Repulsion Energy: ', nuclear_repulsion_energy)

Number Of Particles:  (1, 1)
Number of Spin Orbitals:  4
Nuclear Repulsion Energy:  0.7199689944489797


Generating The Hartree Fock state $|\Phi_o>$

In [22]:
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)

Obtaining the Hartree Fock energy SCF energy $<\Phi_o|\hat{H}|\Phi_o>$

In [23]:
electronic_energy = expectation(qubit_op, init_state)
total_energy = electronic_energy+nuclear_repulsion_energy
print(total_energy)

-1.1169989967540035
